In [3]:
from data import * #1.8s to import & run all codes in that package

In [38]:
import numpy as np
import pandas as pd
import unidecode
import json
import util
import os

In [5]:
data = pd.read_csv(r"data/total.csv", low_memory=False)

SBD = data['SBD']
diem = data.drop(columns=['SBD', 'Tên', 'Ngày Sinh', 'Giới tính'])

info = pd.DataFrame([diem.median(), diem.mode().iloc[0], diem.mean(numeric_only=True).round(2)], index=('Median', 'Mode', 'Mean'))
subjects = tuple(diem.columns)
subjects_lower = tuple([unidecode.unidecode(sj).lower() for sj in subjects])

# Load file geojson chua cac tinh tren ban do VN
with open(r'data/diaphantinh.geojson', encoding='utf8') as f:
    vn_map = json.load(f)
    
# DataFrame chua ma so giao duc va ten tinh
tinh = []

# Chia cac mien:

## Lay ma so giao duc cua cac tinh o moi mien
id_bac = (8, 13, 62, 23, 7, 14, 5, 6, 11, 10, 9, 12, 15, 18, 17, 19, 24, 1, 21, 3, 22, 25, 27, 26, 16)
id_tru = (28, 29, 30, 31, 32, 33, 4, 34, 35, 37, 39, 41, 45, 47, 36, 38, 40, 63, 42)
id_nam = (43, 44, 48, 46, 52, 2, 49, 50, 51, 56, 57, 58, 54, 64, 59, 60, 61, 55, 53)

## Chia cac tinh vao cac mien
bac = []
trung = []
nam = []

for val in vn_map['features']:
    id_ = val['properties']["id"]
    name = val['properties']["ten_tinh"]
    pack = [id_, name]
    
    tinh.append(pack)
    if id_ in id_bac:
        bac.append(pack)
    elif id_ in id_tru:
        trung.append(pack)
    else:
        nam.append(pack)

tinh, bac, trung, nam = map(pd.DataFrame, [tinh, bac, trung, nam])

bac.to_pickle(r'data/map/bac.gz')
trung.to_pickle(r'data/map/trung.gz')
nam.to_pickle(r'data/map/nam.gz')
tinh.to_pickle(r'data/map/tinh.gz')

In [37]:
diem['Văn'].unique()

array([ 8.5 ,  7.5 ,  2.  ,  5.25,  6.5 ,  7.25,  6.75,  6.  ,  7.75,
        8.  ,  6.33,  8.25,  6.25,  5.58,  8.75,  7.  ,  9.  ,  9.25,
        5.75,  3.5 ,  3.25,  4.5 ,  6.42,  5.5 ,  3.75,  2.25,   nan,
        9.5 ,  4.25,  5.  ,  4.75,  3.  ,  2.75,  1.5 ,  2.5 ,  1.75,
        4.  ,  5.33,  7.46,  5.42,  5.9 ,  1.25,  7.33,  6.83,  7.58,
        6.08,  5.91,  0.75,  6.41,  0.  ,  5.92,  6.92,  0.5 ,  7.08,
        6.27,  9.75,  4.83,  5.46,  7.92,  7.83,  6.67,  6.17,  7.67,
        8.33,  6.58,  7.42,  5.66,  3.67,  4.66,  5.83,  5.17,  4.67,
        4.42,  7.16,  8.42,  8.08,  4.08,  2.83,  1.  ,  3.17,  3.58,
        5.41,  6.16,  6.73,  4.92,  5.08,  5.77,  2.92,  2.67,  6.91,
        7.41,  7.91,  7.4 ,  8.58,  8.1 ,  5.67,  4.33,  3.83,  3.91,
        3.33,  5.68,  7.17,  6.66,  4.58,  8.17,  7.34,  4.16,  4.41,
        5.6 ,  4.91,  6.3 ,  5.18,  5.3 ,  7.3 ,  6.7 ,  5.7 ,  5.4 ,
        2.8 ,  6.4 ,  5.16,  7.47,  4.8 ,  7.6 ,  3.9 ,  6.2 ,  7.12,
        7.8 ,  6.9 ,

In [86]:
for id_ in tinh[0]:
    # Tao thu muc chua data cua tinh
    path = f"data/{id_}"

    try:
        os.mkdir(path)
    except FileExistsError:
        pass

    diem_tinh = util.diem_theo_tinh(id_, diem, SBD)

    for subject in diem_tinh.columns:

        # Create a list of tuple to store score and the number of people that have higher score than that
        temp_score_storage = [] # Only temporary cuz we need to serialize it later
        
        for score in diem_tinh[subject].unique():
            temp_score_storage.append((score, util.count_cao_hon_hoac_bang(muc_diem = score, df_diem=diem_tinh, mon = subject, percent=False)))
        if not temp_score_storage: # No record
            temp_score_storage.append((10, 0))

        pd.DataFrame(temp_score_storage, columns=('muc_diem', 'count')).sort_values(by = 'muc_diem').dropna().to_pickle(path = f'{path}/{subjects_lower[subjects.index(subject)]}.gz')

In [87]:
df = pd.read_pickle('data/29/van.gz')

In [88]:
df

,muc_diem,count
0,10,0


In [78]:
df['count'].loc[df['muc_diem'] >= 9].max()

16    2837
17     699
27     135
55       7
Name: count, dtype: int64